In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

In [ ]:
# 공개 상담 미원 조회 & 완료된 민원
def move_to_open_minwon():
    # '공개 상담 민원 조회' 클릭
    driver.execute_script('fnPostLink("/gov/mogaha/ntis/web/emwp/cns/action/EmwpCnslWebAction","selectCnslWebPage","EMWPCnslWebInqL","EmwpCnslWebEJB","selectCnslWebPage","link")')

    # '처리 완료' 만 필터링
    from selenium.webdriver.support.select import Select

    select = Select(driver.find_element(By.NAME, 'pt_deal_state'))
    select.select_by_value("9")
    driver.execute_script('fncSearch()')

    driver.implicitly_wait(300)

In [ ]:
# 결과를 담을 데이터프레임 생성
def make_df():
    cols = ['req_id', 'req_title', 'req_p', 'req_date', 'req_content', 'resp_dept', 'resp_date', 'resp_writer', 'resp_content', 'page_num']
    result_df = pd.DataFrame(columns=cols)
    return result_df

In [ ]:
# 스크랩핑
def scrape(result_df):
    # 가장 끝까지 가서 최대 페이지 수 반환
    driver.find_elements(By.CLASS_NAME, 'navi')[-1].click()
    last_page_num = int(driver.find_elements(By.CLASS_NAME, 'navi')[-3].text)
    print('마지막 페이지 번호 : ', last_page_num)

    # 다시 첫 페이지로 이동
    driver.find_element(By.XPATH, '//*[@id="navigator"]/a[1]').click() # 항상 처음 있는거니까 xpath로 넣어도 됨

    while True :
        # 현재 페이지 번호
        page_num = int(driver.find_elements(By.TAG_NAME, 'span')[-4].text)
        # print('현재 페이지 번호', page_num)

        # 페이지에 있는 민원 개수
        list_num = len(driver.find_element(By.TAG_NAME, 'tbody').find_elements(By.TAG_NAME, 'tr'))

        for i in range(list_num):
            # 현재진행상황 키워드 뽑기 (이송이첩)
            transfer = driver.find_element(By.CSS_SELECTOR, f"#dataSetTb > table > tbody > tr:nth-child({i+1}) > td.td-answer")
            transfer_text = transfer.text

            # 민원 테이블
            table_elem = driver.find_element(By.CLASS_NAME, 'table').find_element(By.TAG_NAME, 'tbody')

            # 하나의 민원 세부 페이지로 들어가기
            # target_minwon은 민원 리스트에 있는 하나의 민원
            target_minwon = driver.find_element(By.TAG_NAME, 'tbody').find_elements(By.TAG_NAME, 'tr')[i].find_element(By.TAG_NAME, 'a')
            if target_minwon.text == '': # 민원 제목이 없으면 세부 페이지로 이동할 수 없음
                pass
            else:
#                 table_elem.find_elements(By.TAG_NAME, 'tr')[i].find_element(By.CLASS_NAME, 'td-list').find_element(By.TAG_NAME, 'a').click()
                target_minwon.click() # 민원 세부 페이지로 이동
                driver.implicitly_wait(100)

                # 답변이 달려있는 경우에는 tbody 태그가 두 개 있음 -> 답변이 달려있는 경우만 데이터 긁기
                if (len(driver.find_elements(By.TAG_NAME, 'tbody')) == 2) or (len(driver.find_elements(By.TAG_NAME, 'tbody')) == 3):
                    # 공개 상담민원(tbody 태그가 두 개 있는데 그 중 첫 번째가 민원 정보가 담겨있는 테이블임)
                    complaint_table = driver.find_elements(By.TAG_NAME, 'tbody')[0].find_elements(By.TAG_NAME, 'tr')
                    # 목록번호
                    req_id = complaint_table[0].find_elements(By.TAG_NAME, 'td')[0].text
                    # 민원 제목
                    req_title = complaint_table[0].find_elements(By.TAG_NAME, 'td')[-1].text
                    # 민원인
                    req_p = complaint_table[1].find_elements(By.TAG_NAME, 'td')[0].text
                    # 작성일
                    req_date = complaint_table[1].find_elements(By.TAG_NAME, 'td')[-2].text
                    # 민원 내용
                    req_content = complaint_table[3].find_elements(By.TAG_NAME, 'td')[-1].text
                    # 첨부파일 거르기
                    r = re.search(r'※ 첨부파일', req_content) # 첨부파일 찾기 -> 모든 새올에서 첨부파일은 '※ 첨부파일'로 되어있음
                    if r:
                        start = r.start()
                        req_content = req_content[:start]

                    # 이송이첩 처리
                    if re.search(r'^(이송이첩|다부처병렬)$', transfer_text):
                        # 이송이첩이라면
                        element = driver.find_element(By.XPATH, '/html/body/main/div/table/tbody/tr/td').text.split('\n')
                        pattern = r'\s*:\s*(.+)'
                        result = re.search(pattern, element[1])

                        resp_date = ""
                        resp_writer = ""
                        resp_call_num = ""
                        resp_content = ""
                        # Check if a match is found
                        if result:
                            resp_dept = result.group(1).strip()
                        else:
                            resp_dept = 'error'
                    else:
                        # 이송이첩이 아니라면 제대로 된 답변 반환
                        # 상담 답변(tbody 두개 중 두 번째꺼가 답변이 담겨있는 테이블임)
                        resp_table = driver.find_elements(By.TAG_NAME, 'tbody')[1].find_elements(By.TAG_NAME, 'tr')

                        # 진짜 답변 테이블일 경우(tr이 3개임)
                        if len(resp_table) == 3:
                            # 담당 부서
                            resp_dept = resp_table[0].find_elements(By.TAG_NAME, 'td')[0].text
                            # 답변 일자
                            resp_date = resp_table[0].find_elements(By.TAG_NAME, 'td')[1].text
                            # 작성자
                            resp_writer = resp_table[1].find_elements(By.TAG_NAME, 'td')[0].text
                            # 전화번호 -> 이건 작성자 직통 번호겠지? 그러면 아마 resp_dept 내에 속한 팀의 전화번호 중 하나일거야
                            resp_call_num = resp_table[1].find_elements(By.TAG_NAME, 'td')[1].text
                            # 답변
                            resp_content = resp_table[2].find_elements(By.TAG_NAME, 'td')[0].text
                        else: # 답변이 아니고 평가인 경우 --> 제목을 테스트라고 넣어서 삭제하기
                            req_title = "테스트"
                            resp_dept = ""
                            resp_date = ""
                            resp_writer = ""
                            resp_call_num = ""
                            resp_content = ""



                    # 가져온 값을 결과 데이터프레임에 추가하기 위해 딕셔너리로 추가
                    data = {
                        'req_id' : req_id,
                        'req_title': req_title,
                        'req_p' : req_p,
                        'req_date' : req_date,
                        'req_content' : req_content,
                        'resp_dept' : resp_dept,
                        'resp_date' : resp_date,
                        'resp_writer' : resp_writer,
                        'resp_content' : resp_content,
                        'page_num' : page_num
                    }

                    # 마지막 행 다음에 추가한 결과 넣기
                    result_df.loc[result_df.shape[0]+1] = data
                    print(page_num, '            ', req_title)

                    # 뒤로가기 -> 다시 목록으로
                    driver.back()
                    driver.implicitly_wait(100)

                else: # tbody가 2개나 3개가 아닌 경우 그냥 뒤로 나가기
                    # 뒤로가기 -> 다시 목록으로
                    print('긁을 수 없다')
                    driver.back()

#                 pass

                # 확인을 위해 가져온 민원 제목만 출력해보기
#                 print(page_num, '            ', req_title)

            # else: # 답변이 없는 민원이라면
                # print(page_num, '            답변이 달려있지 않음')



        # 반복을 계속 할지 여부 확인
        if page_num == last_page_num: # 현재 페이지 번호가 최대 페이지 번호와 같으면
            driver.close() # 창 닫기
            break

        # 다음 페이지로 이동
        driver.find_elements(By.CLASS_NAME, 'navi')[-2].click()

    return result_df

In [ ]:
# 테스트 데이터 지우기 - 제목에 테스트 혹은 test 라는 글이 있음
def remove_test(result_df):
    try:
        drop_idx = result_df[result_df['req_title'].str.contains('테스트|test')].index
        result_df.drop(drop_idx, inplace=True)
        result_df.reset_index(drop=True, inplace=True) # 인덱스 초기화
    except:
        pass
    return result_df

In [ ]:
# url 25개 구 추가
url_dict = {
    '서초구': 'https://eminwon.seocho.go.kr/emwp/gov/mogaha/ntis/web/emwp/cmmpotal/action/EmwpLinkAction.do',
    '강남구': 'https://gangnam.eminwon.seoul.kr/emwp/gov/mogaha/ntis/web/emwp/cmmpotal/action/EmwpMainMgtAction.do',
    '마포구': 'https://eminwon.mapo.go.kr/emwp/gov/mogaha/ntis/web/emwp/cmmpotal/action/EmwpMainMgtAction.do',
    '도봉구': 'https://dobong.eminwon.seoul.kr/emwp/gov/mogaha/ntis/web/emwp/cmmpotal/action/EmwpMainMgtAction.do',
    '성동구': 'https://eminwon.sd.go.kr/emwp/gov/mogaha/ntis/web/emwp/cmmpotal/action/EmwpMainMgtAction.do',
    '광진구': 'https://gwangjin.eminwon.seoul.kr/emwp/gov/mogaha/ntis/web/emwp/cmmpotal/action/EmwpLinkAction.do'
}

In [ ]:
for k, v in url_dict.items():
    print('############# {} #############'.format(k))
    driver = webdriver.Chrome()
    driver.get(v)
    driver.implicitly_wait(300)

    move_to_open_minwon() # 공개 상담 미원 조회 & 완료된 민원
    df = make_df() # 결과를 담을 데이터프레임 생성
    df = scrape(df) # 결과 확인
    final_df = remove_test(df) # 테스트 데이터 지우기

    final_df.to_csv('../data/rowdata/새올/{}.csv'.format(k), index=False, encoding='utf-8-sig')

############# 강남구 #############
마지막 페이지 번호 :  459
1              양재천 너구리(?) 출현
1              2415번 버스기사님 칭찬합니다.
1              통신판매업 신고 - 여러 사이트의 경우
1              그랜드 인터컨티넨탈 서울 파르나스 앞 인도 건널목 교통정리 불법 여부
1              헌릉로(서울 강남구 세곡동 79-6)도로 안전지대 조정 요청
1              도로 노면 홈으로 인한 물튀김 피해
1              실내 공기질 측정기 대여 서비스
1              이동약자의 이동공간 확보를 도와주세요
1              도보위 불법 광고판
1              과태료부과 프로세스 개선요청
1              강남페이 구입이 왜이리 어려운건가요
1              벼룩파리 방역 요청
1              개포동 도시형 생활주택 공사 관련 민웡
1              갈매기공원 도랑 수리 등
1              치과교정과 전문의 허위약력 불법의료광고 신고 (7월3일 민원에 내용추가)
1              단속 공무에 대한 감사 청구 - 강남구청장 감사담당관
1              킥보드 배치로 인한 불편사항
1              디에이치자이개포 상가 가게들의 무질서함으로 장애인 피해, 어린이 부상
1              주정차 단속 24시간 신고제 개정을 요청합니다
1              아파트 중계기 설치 문제
2              대모산입구역 5번출구 공공보행통로에서의 지속적 흡연으로 인한 간접흡연
2              자동차 통행
2              도로 정비 요청
2              가로등 불빛을 가리는 나무를 전지, 전정해 주세요
2              잠시주차구역 주차요금 징수문의
2              절벽놀이터 시설노후화 문

11              온열의자 가동되나요?
11              선정릉 옆 도로공사 대책이 필요합니다
12              소형 폐가전 수거 관련 문의
12              수서동 도서관부지를 해결 해주십시요.
12              불법 현수막 부착에 따른 민원
12              주무관님 칭찬합니다
12              동네 방역 요청
12              개인빌딩 내 이륜차(스쿠터) 주차 거부
12              주차장 공사를 위한 장비 주차 협조 요청의 건
12              종량제쓰레기와 재활용 플라스틱 교환
12              강남스포츠문화센터 오전수영 중급(매일반) 정원 증원 요청 건
12              빌딩앞 스쿠터 및 오토바이 주차금지
12              미성년자 코로나 격리해제 확인서 발급
12              달터근린공원 조성사면 붕괴 및 낙석사고 관련 피해배상 건
12              모기때문에 죽겠습니다
12              개포4동 바퀴벌레 모기 방역 좀 해주세요ㅠ
12              대치동 구마을1지구 공사 소음이 너무 심각합니다.
12              공용부분에 대한 쓰레기 적체로 강남구민이 위험하다.
12              모기 방역요청
12              오늘 오전 10시 즈음 개포시장 쪽 살충 작업에서 살충제를 맞았습니다
12              대치동 구마을 재건축으로 인한 소음과 진동 피해 심각
12              강남구 신사동 588-16 현대썬앤빌 외벽에 대형 고화질 LED 광고판 및공해
13              논현동 62-2 포도더블랙 공사현장 공사소음 민원제가
13              동호대교에서 을지병원 사거리 방향 유턴 차선 신설 후 좌회전 교통 체증
13              신축건물 공사장에서 대지경계 침범하여 담장을 무단으로 설치합니

22              측백나무 구입 및 생활쓰레기 버리기
22              보도블럭 불량 낙상사고 관련 배상 책임회피 민원
22              안심콜 신청
22              음식에 이물질
22              역삼1동 주민센터 주말 오전에 보수공사 앞으로 한번만 더 하면 각오하시기 바랍니다.
22              1가구2주택
22              해외예방접종완료자 보건소시스템예방접종증명서등록
22              온열의자가 너무 뜨겁습니다
22              부설주차장 설치제한 지역(주차상한제 지역) 여부 확인요청드립니다.
22              자곡동 신세계건설 공사 소음 및 주위 환경에 대한 불편
22              건물 철거 문제
22              공동추택 주차장 부족
22              서울시 강남구 신사동 519-23번지 공사차량으로 인한 차량 진출입 불가
22              못골한옥도서관 안내업무 불친절
22              구입한 버터에서 시커먼 곰팡이가 엄청 피어있습니다. 식품위생관련 처분을 바랍니다.
22              도로 파손으로 인한 상해
22              밀미리 경로당에서 달터근린공원으로 가는 입구
22              소장님 박한성 피부과의사 개업일자와폐업일자가적힌서류좀만들어주세요
22              도곡동 힐스테이트 갤러리 지목이 답인 이유는?
22              헌릉로571길20 사거리 신호등 민원
23              수능 15일 남은시점에 재수생들 공부하는 공간 앞 도로 공사를 지금해야 하나요?
23              강남구에서도 내집 주차장 설치비 지원사업을 시행하였으면
23              사유지 내의 무단 석축배수로 정정및 토지 사용료 지급 신청
23              코로나 자가격리대상자인데 강남구에서 정보 이관이 안되고 있습니다

32              미세먼지 프리존 노숙자 관리 및 코로나 확산의 우려
32              도로보수 요청
32              코로나 자가격리대상 선정 기준 관련 의문사항
32              도로공사
32              강남역 살인사건 추모사 개최를 반대합니다
32              세곡동 재활병원 신축공사 인도위 건축자재
32              파리크라상 삼성역키친점(케이크 안에 머리카락)
32              역삼동834-26 전력연구원 공사현장 흙먼지. 공사먼지처리
32              세곡동 재활병원 신축공사 현장 도로위 적재물
32              세곡동 재활병원 신축공사장 인도위 건축자재
32              양재천 수질개선
32              공사 소음 및 진동 신고
32              제발 등교시간은 피해서 작업해주세요
32              주거지역 확성마이크를 사용한 시위
32              노마스크 강남구대책를 듣고 싶네요.
32              불법주정차를 단속하여 주십시오. (역삼문화공원 주차장 공사장 입구 부근 등)
32              국기원
33              개포문화공원 흙길
33              아파트 엘리베이터가 한달넘게 가동되지 않아요
33              테헤란로 선릉역 부근 시위 소음으로 인한 스트레스
33              불법 주정차위반 단속바랍니다
33              아침 8시부터 오후 5시까지 아파트전체가 쿵쿵거리는 소음
33              대청중학교는 늘벗공원에서 체육수업을 중단하라
33              세영팔레스 옆 대우푸르지오 서밋 신축공사
33              영업장이 아닌곳에서 천막을 치고 자판을 깔아놔서 실제 영업장처럼영업하니정리요청함
33              영업장이 아닌곳에서 천막을 치고 자판을 깔아놔서 실제 

42              제1차 중소기업 육성기금 융자지원 신청 자격 관련한 건의
42              강남구 역삼동 헬스장 민원접수 합니다.
42              강남구 도곡동 907-54 공사현장에서 인도구간으로 1 M 점거 훼손하여 빔 설치
긁을 수 없당
42              남부순환로 365길 인도설치 요청
42              양재전화국 사거리 보행 불편
42              파킹 프렌즈 공유 시간
42              사람을찿습니다....?
42              (긴급요청)압구정로2길 일대 염화칼슘미리 뿌려주세요 !
42              강남구립요양병원을 코로나 환자 전용병원으로 지정하는 것 절대 반대
43              강남구립행복요양병원의 코로나전담요양병원 전환 협조요청 철회 강력 촉구
43              12월 무급휴직자 고용안정지원금
43              화재예방을 위한 시설변경요청
43              세곡동행복요양병원코로나전담병원반대
43              안전을 위한 행정지도 요청
43              화재예방을 위한 시설변경요청
43              화재 예방을 위한 시설 변경 요청
43              화재로부터 안전한 시설 변경 요청
43              포스코사거리 지하보도 건의
43              행복요양병원의 코로나 전담병원 지정 반대를 합니다.
43              강남구립행복 요양병원 코로니 전담병원 지정을 반대합니다
43              강남구립 행복요양병원 코로나 전담병원 지정을 반대합니다.
43              언북초~언북어린이집 주변 도보 제설 안되었습니다
43              강남 구립 행복요야병원을 코로나 전담 병원으로 운영한다니 말이 되는건가요?
43              강남구립행복요양병원-코로나 전담 병원 지정 추진 관련
43          

...

426              여권 발급 관련 문의 드립니다
426              도선어린이집 재위탁 관련
426              도선어린이집의 앞날은...
426              마장2지구 재개발추진위원회 위원장은 누구며? 진행상태? 가 궁금합니다.
426              금호2가동 재개발 제13구역
426              지하철 공사 소음 개선 요구
426              도선어린이집재위탁건
426              도선어린이집에 대한 성실한 답변을 듣고 싶습니다!
426              구청대학생아르바이트관련
426              1회용에 관하여 문의드립니다.
426              버스전용차선 단속기준은?
426              도선어린이집 위탁관련사항
426              금호 4가 도로와 금남시장주변의 인도는 어디에
426              성수1가2동 상원 버스정류장 옆 삼거리 교통신호위반 문제
긁을 수 없당
426              등록번호 1790 상원길 초등학생 통학로 확장 및 지중화건 처리는 하는건가요????
426              (주택과) 재개발 현황표
426              담배꽁초 무단투기 단속에 대하여
427              도로의 유실
427              도선어린이집의 앞날...
427              재활용품 수거원들 그날기분에 따라 골라잡아
427              도선어린이집 위탁관련
427              도선어린이집 왜 이렇게 되었습니까?
427              도선어린이집을 위하는 사람은 진정 누구입니까?
427              금호초등학교 주 정차 건의건 시정에 대해서
427              아파트 윗층 건축내부구조변경에 대한 민원
427              음식물 쓰레기 좀 가져가세요.
427              도선어린이집 뒤통수 맞았어

437              불법 주차 문제
437              쓰레기 처리 관련하여
437              너무 심하네요!
437              교통행정과 민원요~
437              민원평가건의
437              성수역 양방향 도로
437              이 무슨 개지랄입니까..
437              송정동(건물)주차 입구 보호시설에 관한건
437              금호동 대현산 배수지 테니스장에 대해서....
437              과태료 처분 납부통지서 납부기한
437              불법주정차에 관해서
437              장애인재활본부가 어떤 단체인지요?
437              금요일 아침에 쓰레기 무단투기 경고문을 붙여주시고 가셨는데..
437              무지개열람실 직원의 불친절
437              대경상고 올라가는길에 있는...
438              지역사회복지 계획안 문의
438              보행도로 보도블럭 경사 수정 요청
438              생태종합 안내도 위치 부적정
438              계단파손
438              성수중학교 쓰레기통 문제
438              민방위 훈련 관련, 주차문제 (8000원)
438              용답동 꽃공원에 관련해서 여쭙겠습니다.
438              살곶이 인라인장 포장
438              대현산 배수지공원의 인라인스케이트장의 위험성
438              주민자치센터
438              쓰레기 무단투기? 난 바보가 아니에요
438              응봉동 개나리길 주차단속 ...
438              서울숲앞 도로진동 해소요청
긁을 수 없당
438              쓰러져 가는 고목(古木) 처리방법 문의
438              쓰레기 무단투기로 

447              응봉동 현대아파트 골목 주차 단속에 대해
447              신호 체계의 문제
448              풍림아파트 북측 도로 표고에 대하여
448              구청장님 및 도시정비국장님 귀하.
448              하왕1-5지구 사업시행관련 허공도로는 누굴위한 도로인가
448              자동차세 연납시 승용차요일제 가입차량의 세금환급 방법 및 확인요청 ?
448              하왕1-5 재개발 신설도로
448              하왕1-5재개발아파트 신설도로 관련 이의제기
448              풍림아파트앞 하왕1-5구역 주택재개발정비구역 도로에 대한 이의 신청
448              성동 우체국앞 주차에 대하여...
448              하왕 1-5구역 도로!
448              농지의 이용의무 위반자 과태료 처분에 관한 질의서
448              민원처리에 좀 실망스럽네요.
448              마장동사무소의 동장님의 답변에 대해서...
448              일관된 탁상행정에 분노하는 민원인!!
448              제설함(염화칼슘) 채워주세요
448              새단장한 달맞이 공원 ..
448              취득세 납부증명서 발급 가능 여부
448              면허세 납부시 통합고지 방법
448              장애인문화기부 릴레이- 책 한권의 행복찾기
448              피시방 면허세에 법이 이상한건지 아니면 구청이 이상한건지
448              국제결혼 문의
449              주차단속
449              존경하는 구청장님께
449              어린이집 보육료감면때문에요..빠른답변부탁드립니다.
449              보행도로 거리 불법 광고 표지판 제거해 주세요!!
449            

459              달콤한 솜사탕의 전도
459              하도 답답해서 질문올립니다. 전세계약후 집에 문제가 있어요.
459              불법주차 단속을 강력히 단속 바랍니다
459              탄원서-인도를 만들어 주세요
459              공영주차장 입찰 실태 에 대해서 말씀드립니다
459              성수동 뚝섬슈퍼의 횡포
459              변경가능한가요
459              성동구내 간판업체개수 문의
459              집앞의 하수도 공사후 처리..
459              가로등 설치 바랍니다..
459              구 소재 대형자동차 주차장이 있는지요?
459              행정서포터즈 등록이 안됩니다.
459              저소득층 전세자금대출 조건을 알고싶습니다.
459              혼인신고에 대해서....
460              아랫글 미안합니다
460              거주자 주차 신경씁시다.
460              차량등록증분실
460              투자유치사업설명회 참석하여주시길바랍니다.
460              주민등록증 재발급에 관해 문의합니다
460              장기 차량 방치 件
460              민방위 교육에 관해서~~
460              새벽에 우는 고양이들 때문에.
460              [도로공사] 후 복구 조속 해결 바랍니다.
460              성동주니어오케스트라에대해서
460              안녕하십니까?
460              자동차관리법 시행령
460              자동차 검사 지연에 따른 과태료 부과의 건
460              정말 시끄러워서 못살겠네요
460              주민등록증때문에...
460              보안등 증설 및 고장신

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=115.0.5790.110)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00F7A813+48355]
	(No symbol) [0x00F0C4B1]
	(No symbol) [0x00E15358]
	(No symbol) [0x00DFD293]
	(No symbol) [0x00E5E37B]
	(No symbol) [0x00E6C473]
	(No symbol) [0x00E5A536]
	(No symbol) [0x00E382DC]
	(No symbol) [0x00E393DD]
	GetHandleVerifier [0x011DAABD+2539405]
	GetHandleVerifier [0x0121A78F+2800735]
	GetHandleVerifier [0x0121456C+2775612]
	GetHandleVerifier [0x010051E0+616112]
	(No symbol) [0x00F15F8C]
	(No symbol) [0x00F12328]
	(No symbol) [0x00F1240B]
	(No symbol) [0x00F04FF7]
	BaseThreadInitThunk [0x76917D59+25]
	RtlInitializeExceptionChain [0x7712B79B+107]
	RtlClearBits [0x7712B71F+191]
